In [5]:
import time
import os
import calendar
import pandas as pd
start = time.time() # старт времени работы программы
count=0
doc=input('Введите путь файла ') # ввод файла
dirname, fname = os.path.split(doc)
df=pd.read_csv(doc, sep='\t', encoding="UTF-8")
df['index'] = df.index
df=df.sort_values(by=['Field', 'Cluster', 'index'])
debit, plotnost, data, debit_vod, debit_gas, plotnost=[], [], [], [], [], []
sum_vod, sum_gas, sum_oil=0.0, 0.0, 0.0
well=df['Name'].values.tolist()
type_skv=df['Well type'].values.tolist()
debit=df['Oil Rate Sm3/day'].values.tolist()
debit_vod=df['Water Rate Sm3/day'].values.tolist()
debit_gas=df['Gas Rate 1000Sm3/d'].values.tolist()
plotnost=df['Oil gravity Kg/m3'].values.tolist()
vvod=df['Drawdown bar'].values.tolist()
data=df['Date'].values.tolist()
Field=df['Field'].values.tolist()
Cluster=df['Cluster'].values.tolist()
nakop_oil=df['Cumulative Oil Production MSm3'].values.tolist()
nakop_water=df['Cumulative Water Production MSm3'].values.tolist()
nakop_gas=df['Cumulative Gas Production MSm3'].values.tolist()
count=len(data)
data.append('99')
debit_f=list(map(float, debit))
plotnost_f=list(map(float, plotnost))
debit_v=list(map(float, debit_vod))
debit_g=list(map(float, debit_gas))
s, st='', ''
t=0
def writeln(parametr, text):
    out_file.write(s)
    out_file.write('\t')
    out_file.write(text)
    out_file.write('\t') 
    for p in range(len(parametr)):
        st=str(parametr[p])
        out_file.write(st)
        out_file.write('\t')
    out_file.write('\n')
    return
def filtr(a):    
    b=[]
    for i in range(len(a)-1):
        for x in [b]:
            if a[i] not in x:
                b.append(a[i])
    return b
Field_filtr=filtr(Field)
Field_filtr.append('Total')
Cust=filtr(Cluster)
a=filtr(data)
god, c=[], []
for i in range(len(a)-1):
    if a[i][6:] not in god:
        god+=[a[i][6:]]
padenie_debitov=[]
nach_debit=0.0
s='temp_padeniya.prn'
doc_out=os.path.join(dirname, s)
out_file=open(doc_out, 'w') # создание файла
out_file.write('Field')
out_file.write('\t')
out_file.write('Claster')
out_file.write('\t')
out_file.write('Name')
out_file.write('\t')
for i in range(len(a)): # запись дат
    out_file.write(a[i])
    out_file.write('\t')
out_file.write('\n')
for i in range(len(well)): # запись данных в таблице
    if i==0 and debit_f[i]==0:padenie=0 # определение дебита прошлого месяца 
    if i==0 and debit_f[i]!=0:padenie=debit_f[i]/debit_f[i]
    if i!=0:    
        if debit_f[i]!=0 and nach_debit==0.0:  # определение начального дебита и падения дебита
            nach_debit=debit_f[i]
        if nach_debit!=0.0: padenie=debit_f[i]/nach_debit
        else: padenie=0.0
    padenie_debitov+=[padenie] # создание таблицы падения дебитов
    if len(a)==len(padenie_debitov): # запись имени пласта, куста скважин и скважины
        out_file.write(Field[i])
        out_file.write('\t')
        out_file.write(Cluster[i])
        out_file.write('\t')
        out_file.write(well[i])
        out_file.write('\t')
        for p in padenie_debitov:
            out_file.write(str(p))
            out_file.write('\t')
        out_file.write('\n')
        nach_debit=0.0
        padenie_debitov=[]
out_file.close()
variant=input('Выберите способ расчета добычи (указать номер способа): 1 - по дебитам скважин, 2 - по накопленным показателям: ')
sum_debit, sum_debit_zh, sum_debit_g, sum_vvod_skv, sum_vvod_nag, sum_vvod_dob, sum_deist_neft, sum_deist_nag, sum_perevod, sum_sdf_dob, sum_sdf_nag=[0.0]*len(god), [0.0]*len(god), [0.0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0.0]*len(god), [0.0]*len(god)
total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod, total_sdf_dob, total_sdf_nag=[0.0]*len(god), [0.0]*len(god), [0.0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0.0]*len(god), [0.0]*len(god)
total_debit_mest, total_debit_zh_mest, total_debit_g_mest, total_vvod_skv_mest, total_vvod_nag_mest, total_vvod_dob_mest, total_deist_neft_mest, total_deist_nag_mest, total_perevod_mest,total_sdf_dob_mest, total_sdf_nag_mest=[0.0]*len(god), [0.0]*len(god), [0.0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0.0]*len(god), [0.0]*len(god)
for j in range(len(Field_filtr)):
    s=Field_filtr[j]+'_year.prn'
    doc_out=os.path.join(dirname, s) # задание пути сохранения результата
    out_file=open(doc_out, 'w') # создание файла
    if Field_filtr[j]!='Total':
        out_file.write('Куст')
        out_file.write('\t')
        out_file.write('Параметр')
        out_file.write('\t')
        for k in range(len(god)): # запись дат
            out_file.write(god[k])
            out_file.write('\t')
        out_file.write('\n')
        for i in range(len(Field)-1): #цикл по строкам
            if Field_filtr[j]==Field[i]:
                if variant=='1':
                    sum_debit[t]=sum_debit[t]+debit_f[i]*plotnost_f[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    total_debit[t]+=debit_f[i]*plotnost_f[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    sum_debit_zh[t]=sum_debit_zh[t]+(debit_f[i]*plotnost_f[i]+debit_v[i]*1000)*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    total_debit_zh[t]+=(debit_f[i]*plotnost_f[i]+debit_v[i]*1000)*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    sum_debit_g[t]=sum_debit_g[t]+debit_g[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]
                    total_debit_g[t]+=debit_g[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]
                else:
                    if well[i]==well[i+1]:
                        sum_debit[t]+=(nakop_oil[i+1]-nakop_oil[i])*1000
                        total_debit[t]+=(nakop_oil[i+1]-nakop_oil[i])*1000
                        sum_debit_zh[t]+=(nakop_oil[i+1]+nakop_water[i+1]-nakop_oil[i]-nakop_water[i])*1000
                        total_debit_zh[t]+=(nakop_oil[i+1]+nakop_water[i+1]-nakop_oil[i]-nakop_water[i])*1000
                        sum_debit_g[t]+=(nakop_gas[i+1]-nakop_gas[i])*1000
                        total_debit_g[t]+=(nakop_gas[i+1]-nakop_gas[i])*1000
                if vvod[i]>=0 and (well[i]!=well[i-1] or vvod[i-1]*0!=0): 
                    sum_vvod_skv[t]+=1
                    total_vvod_skv[t]+=1
                if vvod[i]>=0 and type_skv[i]=='NNS' and (well[i]!=well[i-1] or vvod[i-1]*0!=0):
                    sum_vvod_nag[t]+=1
                    total_vvod_nag[t]+=1
                if vvod[i]>=0 and type_skv[i]=='GS' and (well[i]!=well[i-1] or vvod[i-1]*0!=0): 
                    sum_vvod_dob[t]+=1
                    total_vvod_dob[t]+=1
                if debit_f[i-1]>0 and debit_f[i]==0 and vvod[i]==0 and well[i]==well[i-1]: 
                    sum_perevod[t]+=1
                    total_perevod[t]+=1
                if data[i][3:5]=='12' and vvod[i]>0 and debit_f[i]>0: 
                    sum_deist_neft[t]+=1
                    total_deist_neft[t]+=1
                if data[i][3:5]=='12' and  vvod[i]>=0 and debit_f[i]==0:
                    sum_deist_nag[t]+=1
                    total_deist_nag[t]+=1
                if debit_f[i]>0 and vvod[i]>0: 
                    sum_sdf_dob[t]+=1/12
                    total_sdf_dob[t]+=1/12
                if vvod[i]>=0 and debit_f[i]==0: 
                    sum_sdf_nag[t]+=1/12
                    total_sdf_nag[t]+=1/12
                if data[i][6:]!=data[i+1][6:]: t+=1
                if t==len(sum_debit): t=0
                if Cluster[i]!=Cluster[i+1]:
                    s=Cluster[i]
                    writeln(sum_debit, 'Добыча нефти, тыс.т.') 
                    writeln(sum_debit_zh, 'Добыча жидкости, тыс.т.') 
                    writeln(sum_debit_g, 'Добыча газа, тыс м3') 
                    writeln(sum_vvod_skv, 'Ввод скв, скв') 
                    writeln(sum_vvod_nag, 'Ввод н/н скважин, скв') 
                    writeln(sum_vvod_dob, 'Ввод гориз-х добыв-х скважин с МГРП, скв') 
                    writeln(sum_deist_neft, 'Действующий фонд нефтяных, скв') 
                    writeln(sum_deist_nag, 'Действующий фонд нагнетательных, скв') 
                    writeln(sum_sdf_dob, 'СДФ нефтяных, скв') 
                    writeln(sum_sdf_nag, 'СДФ нагнетательных, скв') 
                    writeln(sum_perevod, 'Переводы в ППД, скв') 
                    sum_debit, sum_debit_zh, sum_debit_g, sum_vvod_skv, sum_vvod_nag, sum_vvod_dob, sum_deist_neft, sum_deist_nag, sum_perevod=[0.0]*len(god), [0.0]*len(god), [0.0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god)
        for p in range(len(god)):
            total_debit_mest[p]+=total_debit[p]
            total_debit_zh_mest[p]+=total_debit_zh[p]
            total_debit_g_mest[p]+=total_debit_g[p]
            total_vvod_skv_mest[p]+=total_vvod_skv[p]
            total_vvod_nag_mest[p]+=total_vvod_nag[p]
            total_vvod_dob_mest[p]+=total_vvod_dob[p]
            total_deist_neft_mest[p]+=total_deist_neft[p]
            total_deist_nag_mest[p]+=total_deist_nag[p]
            total_perevod_mest[p]+=total_perevod[p]
            total_sdf_dob_mest[p]+=total_sdf_dob[p]
            total_sdf_nag_mest[p]+=total_sdf_nag[p]
    else: total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod, total_sdf_dob, total_sdf_nag=total_debit_mest, total_debit_zh_mest, total_debit_g_mest, total_vvod_skv_mest, total_vvod_nag_mest, total_vvod_dob_mest, total_deist_neft_mest, total_deist_nag_mest, total_perevod_mest,total_sdf_dob_mest, total_sdf_nag_mest
    out_file.write('\n')
    out_file.write('Пласт')
    out_file.write('\t')
    out_file.write('Параметр')
    out_file.write('\t')
    for k in range(len(god)): # запись дат
        out_file.write(god[k])
        out_file.write('\t')
    out_file.write('\n')
    s=Field_filtr[j]
    writeln(total_debit, 'Добыча нефти, тыс.т.') 
    writeln(total_debit_zh, 'Добыча жидкости, тыс.т.') 
    writeln(total_debit_g, 'Добыча газа, тыс м3') 
    writeln(total_vvod_skv, 'Ввод скв, скв') 
    writeln(total_vvod_nag, 'Ввод н/н скважин, скв') 
    writeln(total_vvod_dob, 'Ввод гориз-х добыв-х скважин с МГРП, скв') 
    writeln(total_deist_neft, 'Действующий фонд нефтяных, скв') 
    writeln(total_deist_nag, 'Действующий фонд нагнетательных, скв') 
    writeln(total_sdf_dob, 'СДФ нефтяных, скв') 
    writeln(total_sdf_nag, 'СДФ нагнетательных, скв') 
    writeln(total_perevod, 'Переводы в ППД, скв') 
    total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod=[0.0]*len(god), [0.0]*len(god), [0.0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god), [0]*len(god)
    out_file.close()
t=0
sum_debit, sum_debit_zh, sum_debit_g, sum_vvod_skv, sum_vvod_nag, sum_vvod_dob, sum_deist_neft, sum_deist_nag, sum_perevod=[0.0]*len(a), [0.0]*len(a), [0.0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a)
total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod=[0.0]*len(a), [0.0]*len(a), [0.0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a)
total_debit_mest, total_debit_zh_mest, total_debit_g_mest, total_vvod_skv_mest, total_vvod_nag_mest, total_vvod_dob_mest, total_deist_neft_mest, total_deist_nag_mest, total_perevod_mest=[0.0]*len(a), [0.0]*len(a), [0.0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a)
for j in range(len(Field_filtr)):
    s=Field_filtr[j]+'_month.prn'
    doc_out=os.path.join(dirname, s) # задание пути сохранения результата
    out_file=open(doc_out, 'w') # создание файла
    if Field_filtr[j]!='Total':
        out_file.write('Куст')
        out_file.write('\t')
        out_file.write('Параметр')
        out_file.write('\t')
        for k in range(len(a)): # запись дат
            out_file.write(a[k])
            out_file.write('\t')
        out_file.write('\n')
        for i in range(len(Field)-1): #цикл по строкам
            if Field_filtr[j]==Field[i]:
                if variant=='1':
                    sum_debit[t]=sum_debit[t]+debit_f[i]*plotnost_f[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    total_debit[t]+=debit_f[i]*plotnost_f[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    sum_debit_zh[t]=sum_debit_zh[t]+(debit_f[i]*plotnost_f[i]+debit_v[i]*1000)*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    total_debit_zh[t]+=(debit_f[i]*plotnost_f[i]+debit_v[i]*1000)*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]/1000000
                    sum_debit_g[t]=sum_debit_g[t]+debit_g[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]
                    total_debit_g[t]+=debit_g[i]*calendar.monthrange(int(data[i][6:]), int(data[i][3:5]))[1]
                else:
                    if well[i]==well[i+1]:
                        sum_debit[t]+=(nakop_oil[i+1]-nakop_oil[i])*1000
                        total_debit[t]+=(nakop_oil[i+1]-nakop_oil[i])*1000
                        sum_debit_zh[t]+=(nakop_oil[i+1]+nakop_water[i+1]-nakop_oil[i]-nakop_water[i])*1000
                        total_debit_zh[t]+=(nakop_oil[i+1]+nakop_water[i+1]-nakop_oil[i]-nakop_water[i])*1000
                        sum_debit_g[t]+=(nakop_gas[i+1]-nakop_gas[i])*1000
                        total_debit_g[t]+=(nakop_gas[i+1]-nakop_gas[i])*1000
                if vvod[i]>=0 and (well[i]!=well[i-1] or vvod[i-1]*0!=0): 
                    sum_vvod_skv[t]+=1
                    total_vvod_skv[t]+=1
                if vvod[i]>=0 and type_skv[i]=='NNS' and (well[i]!=well[i-1] or vvod[i-1]*0!=0):
                    sum_vvod_nag[t]+=1
                    total_vvod_nag[t]+=1
                if vvod[i]>=0 and type_skv[i]=='GS' and (well[i]!=well[i-1] or vvod[i-1]*0!=0): 
                    sum_vvod_dob[t]+=1
                    total_vvod_dob[t]+=1
                if debit_f[i-1]>0 and debit_f[i]==0 and vvod[i]==0 and well[i]==well[i-1]: 
                    sum_perevod[t]+=1
                    total_perevod[t]+=1
                if debit_f[i]>0 and vvod[i]>0: 
                    sum_deist_neft[t]+=1
                    total_deist_neft[t]+=1
                if vvod[i]>=0 and debit_f[i]==0:
                    sum_deist_nag[t]+=1
                    total_deist_nag[t]+=1
                if data[i]!=data[i+1]: t+=1
                if t==len(a): t=0
                if Cluster[i]!=Cluster[i+1]:
                    s=Cluster[i]
                    writeln(sum_debit, 'Добыча нефти, тыс.т.') 
                    writeln(sum_debit_zh, 'Добыча жидкости, тыс.т.') 
                    writeln(sum_debit_g, 'Добыча газа, тыс м3') 
                    writeln(sum_vvod_skv, 'Ввод скв, скв') 
                    writeln(sum_vvod_nag, 'Ввод н/н скважин, скв') 
                    writeln(sum_vvod_dob, 'Ввод гориз-х добыв-х скважин с МГРП, скв') 
                    writeln(sum_deist_neft, 'Действующий фонд нефтяных, скв') 
                    writeln(sum_deist_nag, 'Действующий фонд нагнетательных, скв') 
                    writeln(sum_sdf_dob, 'СДФ нефтяных, скв') 
                    writeln(sum_sdf_nag, 'СДФ нагнетательных, скв') 
                    writeln(sum_perevod, 'Переводы в ППД, скв') 
                    sum_debit, sum_debit_zh, sum_debit_g, sum_vvod_skv, sum_vvod_nag, sum_vvod_dob, sum_deist_neft, sum_deist_nag, sum_perevod=[0.0]*len(a), [0.0]*len(a), [0.0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a)
        for p in range(len(a)):
            total_debit_mest[p]+=total_debit[p]
            total_debit_zh_mest[p]+=total_debit_zh[p]
            total_debit_g_mest[p]+=total_debit_g[p]
            total_vvod_skv_mest[p]+=total_vvod_skv[p]
            total_vvod_nag_mest[p]+=total_vvod_nag[p]
            total_vvod_dob_mest[p]+=total_vvod_dob[p]
            total_deist_neft_mest[p]+=total_deist_neft[p]
            total_deist_nag_mest[p]+=total_deist_nag[p]
            total_perevod_mest[p]+=total_perevod[p]
    else: total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod=total_debit_mest, total_debit_zh_mest, total_debit_g_mest, total_vvod_skv_mest, total_vvod_nag_mest, total_vvod_dob_mest, total_deist_neft_mest, total_deist_nag_mest, total_perevod_mest
    out_file.write('\n')
    out_file.write('Пласт')
    out_file.write('\t')
    out_file.write('Параметр')
    out_file.write('\t')
    for k in range(len(a)): # запись дат
        out_file.write(a[k])
        out_file.write('\t')
    out_file.write('\n')
    s=Field_filtr[j]
    writeln(total_debit, 'Добыча нефти, тыс.т.') 
    writeln(total_debit_zh, 'Добыча жидкости, тыс.т.') 
    writeln(total_debit_g, 'Добыча газа, тыс м3') 
    writeln(total_vvod_skv, 'Ввод скв, скв') 
    writeln(total_vvod_nag, 'Ввод н/н скважин, скв') 
    writeln(total_vvod_dob, 'Ввод гориз-х добыв-х скважин с МГРП, скв') 
    writeln(total_deist_neft, 'Действующий фонд нефтяных, скв') 
    writeln(total_deist_nag, 'Действующий фонд нагнетательных, скв') 
    writeln(total_sdf_dob, 'СДФ нефтяных, скв') 
    writeln(total_sdf_nag, 'СДФ нагнетательных, скв') 
    writeln(total_perevod, 'Переводы в ППД, скв') 
    total_debit, total_debit_zh, total_debit_g, total_vvod_skv, total_vvod_nag, total_vvod_dob, total_deist_neft, total_deist_nag, total_perevod=[0.0]*len(a), [0.0]*len(a), [0.0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a), [0]*len(a)
    out_file.close()
end = time.time()
print('Время расчета: ')
print(end - start)

Введите путь файла C:\Users\alexa\Desktop\Газпромнефть\проверка\Well.prn
Выберите способ расчета добычи (указать номер способа): 1 - по дебитам скважин, 2 - по накопленным показателям: 2
Время расчета: 
12.895131826400757
